In [3]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import transforms,datasets
from torch.utils.data import Dataset,DataLoader,random_split

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

## 数据集：多种肺炎X光图像

### 数据集

In [4]:
data_dir = "../data/COVID_Dataset"
TRAIN = 'train'
TEST = 'test'
VAL = 'val'

def apply_transform(mode=None):
    size = (299,299)
    crop = 299
    if mode == 'train':
        transform = transforms.Compose([transforms.Resize(size),
                               transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation((-20,+20)),
                               transforms.CenterCrop(crop),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                              ])

    elif mode == 'test' or mode == 'val':
        transform = transforms.Compose([transforms.Resize(size),
                               transforms.CenterCrop(crop),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                              ])

    return transform

trainset = datasets.ImageFolder(os.path.join(data_dir, TRAIN),
                                transform = apply_transform(TRAIN))

trainset,valset = random_split(trainset,[len(trainset)-int(len(trainset)*0.1),int(len(trainset)*0.1)])

testset = datasets.ImageFolder(os.path.join(data_dir, TEST),
                               transform = apply_transform(TEST))

train_loader = DataLoader(trainset,
                          batch_size=50,
                          shuffle=True)

val_loader = DataLoader(valset,
                        batch_size=10)

test_loader = DataLoader(testset,
                         batch_size=1)

len(train_loader),len(test_loader),len(val_loader)




(439, 2706, 244)

### 模型

In [5]:
def get_deepLavV3(out_ch):
    model = torchvision.models.inception_v3(pretrained=True)
    # 冻结参数
    for param in model.parameters():
        param.requires_grad = False
    in_features = model.fc.in_features
    classifier = nn.Sequential(
        nn.Linear(in_features, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, out_ch),
        nn.LogSoftmax(dim=1)
    )
    model.fc = classifier
    return model

# model = get_deepLavV3(2)
# print(model)
# x = torch.tensor(np.random.random((2,3,299,299))).float()
# y,_ = model(x)
# y.shape,_.shape


### 评价指标

In [6]:
def accuracy(preds, labels):
    preds = torch.exp(preds)
    top_p,top_class = preds.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

### 冻结训练

In [5]:
model = get_deepLavV3(4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

epochs = 25
val_loss_min = np.Inf
max_e = 10

# 模型存放路径
model_path = os.path.join('./model')
name = "InceptionV3_pre"
model = model.to(DEVICE)
for epoch in range(epochs):

    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0.0
    val_acc = 0.0

    model.train()
    for images,labels in tqdm(train_loader):
        optimizer.zero_grad()
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        preds,_ = model(images)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(preds, labels)

    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)

    model.eval()
    with torch.no_grad():
        for images,labels in tqdm(val_loader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            loss = criterion(preds, labels)
            val_loss += loss.item()
            val_acc += accuracy(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)

    schedular.step(avg_val_loss)

    print("Epoch : {} \ntrain_loss : {:.6f}, \tTrain_acc : {:.6f}, \nVal_loss : {:.6f}, \tVal_acc : {:.6f}".format(epoch + 1,
                                                                                                                   avg_train_loss, avg_train_acc,
                                                                                                                   avg_val_loss, avg_val_acc))
    if avg_val_loss <= val_loss_min:
        print('Validation loss decreased from ({:.6f} --> {:.6f}).\nSaving model ...'.format(val_loss_min, avg_val_loss))
        torch.save(model,os.path.join(model_path,f"{name}.pth"))
        val_loss_min = avg_val_loss
        max_e = 20
    max_e -= 1
    if max_e<=0:
        break


100%|██████████| 244/244 [00:24<00:00,  9.87it/s]


Epoch : 1 
train_loss : 0.892395, 	Train_acc : 0.631608, 
Val_loss : 0.638627, 	Val_acc : 0.755328
Validation loss decreased from (inf --> 0.638627).
Saving model ...


100%|██████████| 244/244 [00:24<00:00,  9.88it/s]


Epoch : 2 
train_loss : 0.771784, 	Train_acc : 0.690380, 
Val_loss : 0.615903, 	Val_acc : 0.778962
Validation loss decreased from (0.638627 --> 0.615903).
Saving model ...


100%|██████████| 244/244 [00:25<00:00,  9.49it/s]


...

Epoch : 23 
train_loss : 0.638755, 	Train_acc : 0.747304, 
Val_loss : 0.620605, 	Val_acc : 0.797268


100%|██████████| 244/244 [00:24<00:00,  9.96it/s]


Epoch : 24 
train_loss : 0.644522, 	Train_acc : 0.745776, 
Val_loss : 0.627244, 	Val_acc : 0.799727


100%|██████████| 244/244 [00:24<00:00, 10.04it/s]

Epoch : 25 
train_loss : 0.636012, 	Train_acc : 0.748581, 
Val_loss : 0.628385, 	Val_acc : 0.798087


In [6]:
val_loss_min

0.5715676234027401

### 解冻参数

In [7]:
model = torch.load(os.path.join(model_path,f"{name}.pth"))

for param in model.Mixed_7c.parameters():
    param.requires_grad = True
for param in model.Mixed_7b.parameters():
    param.requires_grad = True
for param in model.Mixed_7a.parameters():
    param.requires_grad = True

# for k,v in model.named_parameters():
#     print(f"{k}: {v.requires_grad}")
# model

### finetune

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

epochs =25
# val_loss_min = np.Inf
max_e = 10

name = "InceptionV3_pre_finetune"
model = model.to(DEVICE)
for epoch in range(epochs):

    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0.0
    val_acc = 0.0

    model.train()
    for images,labels in tqdm(train_loader):
        optimizer.zero_grad()
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        preds,_ = model(images)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(preds, labels)

    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)

    model.eval()
    with torch.no_grad():
        for images,labels in tqdm(val_loader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            loss = criterion(preds, labels)
            val_loss += loss.item()
            val_acc += accuracy(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)

    schedular.step(avg_val_loss)

    print("Epoch : {} \ntrain_loss : {:.6f}, \tTrain_acc : {:.6f}, \nVal_loss : {:.6f}, \tVal_acc : {:.6f}".format(epoch + 1,
                                                                                                                   avg_train_loss, avg_train_acc,
                                                                                                                   avg_val_loss, avg_val_acc))
    if avg_val_loss <= val_loss_min:
        print('Validation loss decreased from ({:.6f} --> {:.6f}).\nSaving model ...'.format(val_loss_min, avg_val_loss))
        torch.save(model, os.path.join(model_path,f"{name}.pth"))
        val_loss_min = avg_val_loss
        max_e = 10
    max_e -= 1
    if max_e<=0:
        break


100%|██████████| 244/244 [00:24<00:00, 10.01it/s]


Epoch : 1 
train_loss : 0.510907, 	Train_acc : 0.806536, 
Val_loss : 0.372776, 	Val_acc : 0.869672
Validation loss decreased from (0.571568 --> 0.372776).
Saving model ...


100%|██████████| 244/244 [00:24<00:00,  9.77it/s]


Epoch : 2 
train_loss : 0.386504, 	Train_acc : 0.854302, 
Val_loss : 0.359843, 	Val_acc : 0.872131
Validation loss decreased from (0.372776 --> 0.359843).
Saving model ...


100%|██████████| 244/244 [00:23<00:00, 10.37it/s]


...

Epoch : 23 
train_loss : 0.103188, 	Train_acc : 0.961800, 
Val_loss : 0.199904, 	Val_acc : 0.924590
Validation loss decreased from (0.204716 --> 0.199904).
Saving model ...


100%|██████████| 244/244 [00:25<00:00,  9.70it/s]


Epoch : 24 
train_loss : 0.101842, 	Train_acc : 0.964146, 
Val_loss : 0.188753, 	Val_acc : 0.925819
Validation loss decreased from (0.199904 --> 0.188753).
Saving model ...


100%|██████████| 244/244 [00:24<00:00, 10.17it/s]

Epoch : 25 
train_loss : 0.101155, 	Train_acc : 0.963327, 
Val_loss : 0.199174, 	Val_acc : 0.922131


In [9]:
val_loss_min

0.18875324098607066

### 测试

In [7]:
name = "InceptionV3_pre_finetune"
model_path = os.path.join('./model')
model = torch.load(os.path.join(model_path,f"{name}.pth"))
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

# 混淆矩阵
M = np.zeros((4,4))
with torch.no_grad():
    test_loss=0
    test_acc=0
    for images,labels in tqdm(test_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        preds = model(images)
        loss = criterion(preds, labels)
        test_loss += loss.item()
        test_acc += accuracy(preds, labels)
        
        M[preds.argmax(),labels[0]]+=1
        
    avg_test_loss = test_loss / len(test_loader)
    avg_test_acc = test_acc / len(test_loader)
    print("Test_loss : {:.6f}, \tTest_acc : {:.6f}".format(avg_test_loss,avg_test_acc))
    
    M = M.astype("int")
    print(M)
    

100%|██████████| 2706/2706 [01:07<00:00, 39.91it/s]

Test_loss : 0.198834, 	Test_acc : 0.929047
[[ 360    3    3    0]
 [   4  764   33   22]
 [   3   65 1132    4]
 [   0   47    8  258]]
